## Embedding a Bokeh server in a Notebook

This notebook shows how a Bokeh server application can be embedded inside a Jupyter notebook. 

In [2]:
'''
import yaml

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure, curdoc
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

output_notebook()
'''

'\nimport yaml\n\nfrom bokeh.layouts import column\nfrom bokeh.models import ColumnDataSource, Slider\nfrom bokeh.plotting import figure, curdoc\nfrom bokeh.themes import Theme\nfrom bokeh.io import show, output_notebook\n\nfrom bokeh.sampledata.sea_surface_temperature import sea_surface_temperature\n\noutput_notebook()\n'

There are various application handlers that can be used to build up Bokeh documents. For example, there is a `ScriptHandler` that uses the code from a `.py` file to produce Bokeh documents. This is the handler that is used when we run `bokeh serve app.py`. In the notebook we can use a function to define a Bokehg application.

Here is the function `bkapp(doc)` that defines our app:

In [3]:
'''
def bkapp(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))
'''

'\ndef bkapp(doc):\n    df = sea_surface_temperature.copy()\n    source = ColumnDataSource(data=df)\n\n    plot = figure(x_axis_type=\'datetime\', y_range=(0, 25),\n                  y_axis_label=\'Temperature (Celsius)\',\n                  title="Sea Surface Temperature at 43.18, -70.43")\n    plot.line(\'time\', \'temperature\', source=source)\n\n    def callback(attr, old, new):\n        if new == 0:\n            data = df\n        else:\n            data = df.rolling(\'{0}D\'.format(new)).mean()\n        source.data = ColumnDataSource.from_df(data)\n\n    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")\n    slider.on_change(\'value\', callback)\n\n    doc.add_root(column(slider, plot))\n\n    doc.theme = Theme(json=yaml.load("""\n        attrs:\n            Figure:\n                background_fill_color: "#DDDDDD"\n                outline_line_color: white\n                toolbar_location: above\n                height: 500\n                width: 8

Now we can display our application using ``show``, which will automatically create an ``Application`` that wraps ``bkapp`` using ``FunctionHandler``. The end result is that the Bokeh server will call ``bkapp`` to build new documents for every new sessions that is opened.

**Note**: If the current notebook is not displayed at the default URL, you must update the `notebook_url` parameter in the comment below to match, and pass it to `show`.